# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', sep=',')

In [2]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [3]:
df.columns = ['Target', 'ID', 'Date', 'Flag', 'User', 'Text']

In [4]:
df = df.sample(frac=0.25)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
import re
def clean_up(text):
    return re.sub(r'[^a-zA-Z]', ' ', text)

In [6]:
df['processed text'] = df['Text'].apply(clean_up)

In [7]:
df.head()

,Target,ID,Date,Flag,User,Text,processed text
1555845,4,2185238157,Mon Jun 15 17:05:11 PDT 2009,NO_QUERY,Cam_STL,@luvnewkids I feel great now! I'm no longer co...,luvnewkids I feel great now I m no longer co...
102072,0,1794780515,Thu May 14 06:32:43 PDT 2009,NO_QUERY,christinebrady,ugh my tum tum hurts is it friday yet? i need...,ugh my tum tum hurts is it friday yet i need...
1434981,4,2060732688,Sat Jun 06 19:28:20 PDT 2009,NO_QUERY,johnny_posh,Thinking of surprising/visiting mike at work- ...,Thinking of surprising visiting mike at work ...
966444,4,1827692104,Sun May 17 11:14:48 PDT 2009,NO_QUERY,Courtney_ATTACK,I get to see Jacob,I get to see Jacob
948347,4,1823387415,Sat May 16 21:50:16 PDT 2009,NO_QUERY,jrkjr,It would be an honor if you joined our Fan Pag...,It would be an honor if you joined our Fan Pag...


In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import nltk
def tok(text):
    return nltk.word_tokenize(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\feren\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\feren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
tokenized = df['Text'].apply(tok)

In [11]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords 

In [12]:
stop_words = set(stopwords.words('english')) 
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

In [15]:
def stem_and_lemm(text):
    text = stemmer.stem(text)
    text = lemmatizer.lemmatize(text)

    return [w for w in text if not w in stop_words] 

In [16]:
df['text_processed'] = df['Text'].apply(stem_and_lemm)

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [24]:
import collections
bagofw = []
for line in df['text_processed']:
    print(line)
    bagofw.append(collections.Counter(re.findall(r'\w+', str(line))))
    

['@', 'l', 'u', 'v', 'n', 'e', 'w', 'k', ' ', ' ', 'f', 'e', 'e', 'l', ' ', 'g', 'r', 'e', ' ', 'n', 'w', '!', ' ', "'", ' ', 'n', ' ', 'l', 'n', 'g', 'e', 'r', ' ', 'c', 'n', 'g', 'u', '~', 'c', 'u', 'l', 'l', ' ', 'b', 'n', ' ', 'l', 'k', 'n', 'g', ' ', ' ', 'v', 'e', 'g', ' ', 'e', 'l', '-', 'h', 'n', 'k', 'n', 'g', ' ', 'b', ' ', 'g', 'n', 'g', ' ', ' ', 'h', 'e', ' ', 'c', 'n', 'c', 'e', 'r', ' ', 'h', 'e', 'r', 'e', '!', ' ', 'k', 'e', '!', ' ']
['u', 'g', 'h', ' ', ' ', 'u', ' ', 'u', ' ', 'h', 'u', 'r', ' ', ' ', ' ', ' ', 'f', 'r', ' ', 'e', '?', ' ', ' ', 'n', 'e', 'e', ' ', 'e', ' ', 'l', 'e', 'e', 'p', '!', '!']
['h', 'n', 'k', 'n', 'g', ' ', 'f', ' ', 'u', 'r', 'p', 'r', 'n', 'g', '/', 'v', 'n', 'g', ' ', 'k', 'e', ' ', ' ', 'w', 'r', 'k', '-', ' ', 'h', 'e', 'n', ' ', 'f', 'f', ' ', ' ', 'n', 'n', 'e', 'r', ' ']
[' ', 'g', 'e', ' ', ' ', 'e', 'e', ' ', 'j', 'c', 'b', ' ']
[' ', 'w', 'u', 'l', ' ', 'b', 'e', ' ', 'n', ' ', 'h', 'n', 'r', ' ', 'f', ' ', 'u', ' ', 'j', 'n', 

['@', 'l', 'f', 'r', ' ', 'b', 'b', '!', ' ', "'", ' ', 'l', 'l', ' ', ' ', 'h', 'e', ',', ' ', 'n', ' ', 'b', 'e', ' ', 'u', 'c', 'k', ' ', 'h', 'e', ' ', 'n', ' ', 'h', 'e', ' ']
[' ', 'n', 'g', ' ', 'r', '.', ' ', ' ', ' ', ' ', ' ', ' ']
['w', 'r', 'k', 'n', 'g', ' ', ' ', '1', 'p', ' ', 'r', 'r', 'w', ' ', ' ', 'n', "'", ' ', 'w', 'n', 'n', ',', ' ', 'h', 'e', ' ', 'u', 'n', '.']
['@', 'c', 'h', 'e', 'l', 'l', 'e', 'e', 'h', ' ', ' ', 'w', ' ', 'v', 'e', 'r', ' ', 'u', 'c', 'h', ' ', ' ', 'n', ' ', 'h', 'u', 'g', 'h', ' ', ' ', 'u', ' ', 'h', 'v', 'e', ' ', 'p', 'l', 'e', 'n', ' ', 'f', ' ', 'v', ',', ' ', 'v', 'h', ' ', ' ', 'c', 'u', 'l', ' ', 'w', 'c', 'h', ' ', 'n', ' ', 'h', 'e', ' ', 'n', ' ', ' ', 'n', ' ', 'f', 'u', 'u', 'r', 'e', '?', ' ']
['@', 'f', 'n', 'n', 'g', 'k', 'r', ' ', 'e', ' ', ' ', 'k', 'r', '!', '!', '!', ' ']
['j', 'u', ' ', 'k', 'n', 'g', '.', ' ', ' ', 'h', 'v', 'e', ' ', ' ', 'w', '.', ' ']
['@', 'b', 'r', 'p', 'b', ' ', 'e', 'p', '.', ' ', 'j', 'u', ' '

['@', 'n', 'h', 'n', 'w', 'l', 'e', 'r', ' ', 'u', 'n', ' ', 'l', 'k', 'e', ' ', ' ', 'g', 'r', 'e', ' ', 'n', 'g', 'h', ' ', ' ', 'e', ' ', ' ', 'h', 'p', 'e', ' ', ' ', 'w', '!', '!']
['@', 'u', 'n', 'g', 'w', ' ', 'g', 'r', 'e', ',', ' ', 'h', ' ', 'l', 'k', ' ', 'u', 'c', 'h', ' ', 'u', 'c', 'h', ' ', 'b', 'e', 'e', 'r', ' ', 'n', 'w', '!', ' ']
['@', 'h', 'e', 'w', '_', 'b', 'r', 'r', ' ', 'w', 'w', 'w', 'w', 'w', ' ', '!', '!', '!', ' ', ' ', 'h', 'p', 'e', ' ', 'h', 'e', ' ', 'e', 'v', 'e', 'n', 'u', 'l', 'l', ' ', 'c', 'e', ' ', 'b', 'c', 'k', ' ', ' ', 'h', 'e', ' ', 'f', 'r', ' ', ' ', 'r', ' ', 'j', 'u', ' ', 'b', 'e', ' ', ' ', 'f', 'r', 'e', 'n', ' ', 'f', 'r', ' ', 'l', 'f', 'e', '?']
['e', 'x', 'c', 'n', 'g', ' ', 'w', 'e', 'e', 'k', ' ', 'h', 'e', '.', '.', ' ', ' ', 'l', 'f', 'e', ' ', 'f', 'e', 'e', 'l', ' ', 'l', 'k', 'e', ' ', ' ', 'v', 'e', ' ']
['g', 'r', 'e', ' ', 'e', 'e', 'n', 'g', ' ', 'p', 'e', 'e', ' ', 'n', ' ', 'h', 'e', ' ', 'r', 'e', ' ', 'f', ' ', 'h', 

['@', 'c', 'r', 'r', 'e', 'l', 'l', ' ', 'w', 'c', 'h', ' ', 'e', ' ', 'e', 'e', ' ', 'z', 'z', 'r', '.', ' ', ' ', 'p', 'r', 'e', 'f', 'e', 'r', 'b', 'l', ' ', 'h', ' ', '&', 'q', 'u', ';', 'r', 'e', 'e', ' ', ' ', 'k', 'l', 'l', '&', 'q', 'u', ';', ' ', 'u', 'f', 'f', '.', '.', '.', ' ', ' ', 'h', 'e', 'l', 'p', ' ', 'e', 'e', ' ', 'h', 'e', ' ', 'b', 'r', 'e']
[' ', 'f', 'r', ' ', 'b', 'n', 'f', 'r', 'e', ' ', 'f', ' ', 'h', 'e', ' ', 'e', 'n', ' ', 'w', ' ', ' ', 'u', 'c', 'c', 'e', ':', ' ', '@', 'n', 'g', 'e', ',', ' ', '@', 'e', 'h', 'n', 'c', 'h', ',', ' ', 'n', ' ', '@', 'j', 'v', 'e', 'l', 'k', 'f', 'f', ' ', 'l', 'l', ' ', 'c', 'e', '.', ' ', 'n', 'w', ',', ' ', 'b', 'e', '-', '-', 'l', 'l', ' ', 'h', 'v', 'e', ' ', 'w', 'r', 'k', ' ', 'r', 'r', 'w', '.']
['r', 'g', 'h', 'h', ' ', ' ', ' ', 'e', 'r', ' ', 'n', ' ', 'w', 'n', ' ', ' ', 'g', 'e', ' ', 'u', ' ', 'f', ' ', 'b', 'e', ' ', 'n', 'e', 'e', ' ', 'r', ' ', 'l', 'e', 'e', 'p', '!', ' ', ' ', ' ', ' ', 'u', ' ']
['w', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
df

,Target,ID,Date,Flag,User,Text,processed text,text_processed
1555845,4,2185238157,Mon Jun 15 17:05:11 PDT 2009,NO_QUERY,Cam_STL,@luvnewkids I feel great now! I'm no longer co...,luvnewkids I feel great now I m no longer co...,"[@, l, u, v, n, e, w, k, , , f, e, e, l, , ..."
102072,0,1794780515,Thu May 14 06:32:43 PDT 2009,NO_QUERY,christinebrady,ugh my tum tum hurts is it friday yet? i need...,ugh my tum tum hurts is it friday yet i need...,"[u, g, h, , , u, , u, , h, u, r, , , , ..."
1434981,4,2060732688,Sat Jun 06 19:28:20 PDT 2009,NO_QUERY,johnny_posh,Thinking of surprising/visiting mike at work- ...,Thinking of surprising visiting mike at work ...,"[h, n, k, n, g, , f, , u, r, p, r, n, g, /, ..."
966444,4,1827692104,Sun May 17 11:14:48 PDT 2009,NO_QUERY,Courtney_ATTACK,I get to see Jacob,I get to see Jacob,"[ , g, e, , , e, e, , j, c, b, ]"
948347,4,1823387415,Sat May 16 21:50:16 PDT 2009,NO_QUERY,jrkjr,It would be an honor if you joined our Fan Pag...,It would be an honor if you joined our Fan Pag...,"[ , w, u, l, , b, e, , n, , h, n, r, , f, ..."
...,...,...,...,...,...,...,...,...
1013207,4,1881293814,Fri May 22 04:13:47 PDT 2009,NO_QUERY,mariakath,it's been a while that I haven't been updated....,it s been a while that I haven t been updated ...,"[', , b, e, e, n, , , w, h, l, e, , h, , ..."
118208,0,1827787552,Sun May 17 11:26:49 PDT 2009,NO_QUERY,EJMcIver,I full on hate maths Darn GCSE's :'(,I full on hate maths Darn GCSE s,"[ , f, u, l, l, , n, , h, e, , h, , , r, ..."
40443,0,1573787553,Tue Apr 21 02:51:32 PDT 2009,NO_QUERY,cielbluangel,I want to go home...,I want to go home,"[ , w, n, , , g, , h, e, ., ., ., ]"
361998,0,2047199892,Fri Jun 05 13:17:52 PDT 2009,NO_QUERY,igortizz,@kirika_96_ I miss you! you dont have time to ...,kirika I miss you you dont have time to ...,"[@, k, r, k, _, 9, 6, _, , , , u, !, , u, ..."


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# your code here

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here